In [236]:
import numpy as np
np.set_printoptions(precision=5, suppress=True)

# Создание и заполнение матрицы

Создаем и заполняем матрицу входными данными из файла __data.txt__. Размерность определяется автоматически.
Предполагается, что данные введены корректно в естественной форме:
$$\begin{bmatrix}
    a_{11} & a_{12} & ... & a_{1n} \\
    a_{21} & a_{22} & ... & a_{2n}\\
    . & . & ... & .\\
    . & . & ... & .\\
    a_{n1} & a_{n2} & ... & a_{nn}
  \end{bmatrix}
 \begin{bmatrix}
    b_1\\
    b_2\\
    .\\
    .\\
    b_n
  \end{bmatrix}$$

In [237]:
data = np.loadtxt("data2.txt", delimiter=' ', dtype=np.float)
data_cpy = np.copy(data)
n = data.shape[0] # размерность
print(data) # для наглядности выведем матрицу

[[ 2.  3. 11.  5.  2.]
 [ 1.  1.  5.  2.  1.]
 [ 2.  1.  3.  2. -3.]
 [ 1.  1.  3.  4. -3.]]


# Реализация модифицированного метода Гаусса

In [238]:
def Gauss(data, n):
    # Прямой ход
    permutations = np.array(range(0, n))
    for i in range(0, n):
        if data[i][i] == 0: # i-й элемент строки нулевой?
            for j in range(i + 1, n):
                if data[j][i] != 0:
                    data[[i, j]] = data[[j, i]] # меняем местами строчки
                    break
        ### Модифицированный метод гаусса (с выбором главного элемента)
        col_max = i + np.argmax(abs(data[i][i:n]))
        data[:,[i, col_max]] = data[:,[col_max, i]]
        permutations[i], permutations[col_max] = permutations[col_max], permutations[i]
        ###
        data[i] /= data[i][i] # делим i-ю строку на её i-й элемент
        for j in range(i + 1, n):
            data[j] -= data[i] * data[j][i] # вычитаем из всех остальных строк i-ю, умноженную на нужное число
    # Обратный ход
    x = data[:, n] # создаем вектор неизвестных, заранее присвоив ему значение столбца свободных членов
    for i in range(n - 1, -1, -1): # последовательно восставливаем ответ
        for j in range(n - 1, i, -1):
            x[i] -= x[j] * data[i][j]
    res = np.array(list(zip(x, permutations))) # массив пар (x, i), где x является i-й компонентой решения системы
    res = res[res[:, 1].argsort()] # сортировка по номеру переменной
    x = res[:, 0] # вытаскиваем вектор решений в естественном порядке (от x_0 до x_n)
    return x # возвращаем решение системы

In [239]:
print(Gauss(data, n))

[-2. -0.  1. -1.]


Сверим найденное решение с решением, полученным с использованием специальной библиотеки Python для решения задач линейной алгебры.

In [240]:
print(np.linalg.solve(data_cpy[:, :n], data_cpy[:, n]))

[-2. -0.  1. -1.]


Как видим, решение полностью совпало.

# Вычисление определителя и обратной матрицы

Для вычисления обратной матрицы присоединим к исходной справа единичную матрицу и будем приводить исходную к единичной, выполняя все преобразования на присоединенной. В итоге присоединенная матрица окажется обратной к исходной. Параллельно вычисляем определитель, перемножая получаемые по ходу преобразований элементы на главной диагонали (предполагается, что определитель ненулевой).

In [241]:
def det_inverse(data_cpy, n):
    det = 1
    data_inv = data_cpy[:, :n]
    id = np.identity(n) # единичная матрица порядка n
    for i in range(0, n): # присоединяем
        data_inv = np.column_stack((data_inv, id[:, i]))
    for i in range(0, n): # приводим "левую" матрицу к нижнитреугольному виду с единицами на диагонали
        if data_inv[i][i] == 0: # i-й элемент строки нулевой?
            for j in range(i + 1, n):
                if data_inv[j][i] != 0:
                    data_inv[[i, j]] = data_inv[[j, i]] # меняем местами строчки
                    break
        det *= data_inv[i][i] # последовательно вычисляем определитель
        data_inv[i] /= data_inv[i][i] # делим i-ю строку на её i-й элемент
        for j in range(i + 1, n):
            data_inv[j] -= data_inv[i] * data_inv[j][i] # вычитаем из всех остальных строк i-ю, умноженную на нужное число
    for i in range(n - 1, -1, -1): # # приводим "левую" матрицу к единичному виду
        for j in range(i - 1, -1, -1):
            data_inv[j] -= data_inv[i] * data_inv[j][i] # вычитаем из всех остальных строк i-ю, умноженную на нужное число
    inverse_matrix = data_inv[:, n:] # вытаскиваем нужную часть
    return det, inverse_matrix

In [242]:
det, inverse_matrix = det_inverse(data_cpy, n)
print(det) # выводим определитель
print(inverse_matrix) # выводим обратную матрицу

14.0
[[-0.28571  0.28571  0.71429 -0.14286]
 [ 1.28571 -2.78571  0.28571 -0.35714]
 [-0.14286  0.64286 -0.14286 -0.07143]
 [-0.14286  0.14286 -0.14286  0.42857]]


Проверим полученные результаты, опять же, с помощью библиотечных функций.

In [243]:
print(np.linalg.det(data_cpy[:, :n])) # определитель
print(np.dot(data_cpy[:, :n], inverse_matrix)) # умножаем исходную матрицу на полученную обратную

14.000000000000004
[[ 1. -0. -0.  0.]
 [ 0.  1. -0. -0.]
 [ 0.  0.  1. -0.]
 [ 0.  0. -0.  1.]]


# Тестирование

Далее будут представлены резульаты тестирование программы на системах линейных алгебраических уравнений, которые были предложены в приложении 1-2. Первая из систем была проверена выше. Ниже приводятся непосредственные ответы, полученные с помощью описанной выше программы, а также с помощью функции __np.linalg.solve__.

In [244]:
# Вторая матрица
data2 = np.loadtxt("data3.txt", delimiter=' ', dtype=np.float)
data2 = data2[:, :data2.shape[0]]
print("%.5f" % np.linalg.det(data2))
print(data2)

-0.00000
[[ 2. -1.  1.  2.]
 [ 6. -3.  2.  4.]
 [ 6. -3.  4.  8.]
 [ 4. -2.  1.  1.]]


Вторая матрица из приложения 1-2 является вырожденной, поэтому рассмотренные выше методы для неё не годятся.

In [255]:
# Третья матрица
data3 = np.loadtxt("data4.txt", delimiter=' ', dtype=np.float)
print(data3)
print(np.linalg.solve(data3[:, :n], data3[:, n]))
print(Gauss(data3, n))
res = det_inverse(data3, n)
print("%.5f" % np.linalg.det(data3[:, :n]))
print("%.5f" % res[0])
print(res[1])
print(inv)
inv = np.linalg.inv(data3[:, :n])
print(np.dot(res[1], data3[:, :n]))
print(np.dot(inv, data3[:, :n]))

[[ 1.  1. -3.  1. -1.]
 [ 2.  1. -2.  0.  1.]
 [ 1.  1.  1.  0.  3.]
 [ 1.  2. -3. -7.  1.]]
[ 0.91667  1.11111  0.97222 -0.11111]
[ 0.91667  1.11111  0.97222 -0.11111]
1.00000
1.00000
[[ 1.       0.33333  0.25     0.25   ]
 [ 0.       1.      -0.25     0.75   ]
 [ 0.       0.       1.      -1.     ]
 [ 0.       0.       0.       1.     ]]
[[ 1.       0.33333  0.25     0.25   ]
 [ 0.       1.      -0.25     0.75   ]
 [ 0.       0.       1.      -1.     ]
 [ 0.       0.       0.       1.     ]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


Как видим, всё совпадает.